In [1]:
import json
import numpy as np
from pprint import pprint
from pathlib import Path

In [2]:
datadir = Path('data/recipe1m/dl')
l1 = json.load(open(datadir / 'layer1.json'))
l2 = json.load(open(datadir / 'layer2.json'))
l2p = json.load(open(datadir / 'layer2+.json'))

In [3]:
l2[0]

{'id': '00003a70b1',
 'images': [{'id': '3e233001e2.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picaYYmb9.jpg'},
  {'id': '7f749987f9.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picpy37SW.jpg'},
  {'id': 'aaf6b2dcd3.jpg',
   'url': 'http://img.sndimg.com/food/image/upload/w_512,h_512,c_fit,fl_progressive,q_95/v1/img/recipes/47/91/49/picX9CNE2.jpg'}]}

In [4]:
nut = json.load(open(datadir / 'recipes_with_nutritional_info.json'))

In [5]:
l1d = {l["id"]: l for l in l1}
l2d = {l["id"]: l for l in l2}
l2pd = {l["id"]: l for l in l2p}

In [6]:
imglist = [l2d.get(e["id"], {'images': []})['images'] for e in nut]

In [7]:
imglistp = [l2pd.get(e["id"], {'images': []})['images'] for e in nut]

In [8]:
len(imglist)

51235

In [9]:
print("num of nut values entries")
len(nut)

num of nut values entries


51235

In [10]:
print("num of nut values entries with images")
sum([int(len(i) > 0) for i in imglist])

num of nut values entries with images


20232

In [11]:
print("num of img2nut entries")
len([e for i in imglist for e in i])

num of img2nut entries


50718

In [12]:
print("above three values for recipe1m+ (including google-scraped images which are only barely related to the recipes)")
print(len(nut))
print(sum([int(len(i) > 0) for i in imglistp]))
print(len([e for i in imglistp for e in i]))

above three values for recipe1m+ (including google-scraped images which are only barely related to the recipes)
51235
50797
718154


In [32]:
# same outlier removal as for our own dataset
def filter_outliers(data: list, *, factor=2, key=lambda x: x):
    l = len(data)
    while True:
        # https://www.kdnuggets.com/2017/02/removing-outliers-standard-deviation-python.html
        vals = [key(e) for e in data]
        mean = np.mean(vals)
        stddev = np.std(vals)
        print("mean", mean)
        print("std", stddev)
        filt_min = mean - factor * stddev
        filt_max = mean + factor * stddev
        data = [
            ele for val, ele in zip(vals, data) if val >= filt_min and val <= filt_max
        ]
        if len(data) == l:
            break
        l = len(data)
    return (data, filt_min, filt_max)

In [33]:
datapoints = [{**l1d.get(e["id"], {}), **l2d.get(e["id"], {"images": []}), "nut": e} for e in nut]

In [34]:
_x = len(datapoints)
datapoints = [d for d in datapoints if len(d["images"]) > 0]
print("removed no img", _x - len(datapoints))

removed no img 31003


In [77]:
pprint(nut[0])

{'fsa_lights_per100g': {'fat': 'green',
                        'salt': 'green',
                        'saturates': 'green',
                        'sugars': 'orange'},
 'id': '000095fc1d',
 'ingredients': [{'text': 'yogurt, greek, plain, nonfat'},
                 {'text': 'strawberries, raw'},
                 {'text': 'cereals ready-to-eat, granola, homemade'}],
 'instructions': [{'text': 'Layer all ingredients in a serving dish.'}],
 'nutr_per_ingredient': [{'fat': 0.8845044000000001,
                          'nrg': 133.80964,
                          'pro': 23.110512399999998,
                          'sat': 0.26535132,
                          'sod': 81.64656,
                          'sug': 7.348190400000001},
                         {'fat': 0.46,
                          'nrg': 49.0,
                          'pro': 1.02,
                          'sat': 0.023,
                          'sod': 2.0,
                          'sug': 7.43},
                         {'fat

AttributeError: 'list' object has no attribute 'contains'

In [54]:
def print_filt(key):
    bef_count = len(datapoints)
    data, filt_min, filt_max = filter_outliers(datapoints, key=key)
    dpid = {d["id"] for d in data}
    outliers = [d for d in datapoints if d["id"] not in dpid]
    print(f"filtering kcal to [{filt_min}, {filt_max}]")
    print(f"outliers: removed {bef_count - len(data)} of {bef_count}")
    print(len(outliers))
    print("final recipe count", len(data))
    print("final data point count", len([i for d in data for i in d["images"]]))
    return outliers

In [55]:
outliers = print_filt(key=lambda r: r["nut"]["nutr_values_per100g"]["energy"])

mean 269.19589095942564
std 177.30687200065913
mean 251.55525678200382
std 155.73749902287216
mean 241.77283498681555
std 146.719412003905
mean 235.91349387040356
std 141.7999674173209
mean 232.18052320346095
std 138.7943937451992
mean 229.83656599102235
std 136.95581040247546
mean 227.81166199772446
std 135.3860411413802
mean 226.31902873462093
std 134.2354668401879
mean 224.85019269291834
std 133.10837353234487
mean 223.5674525290661
std 132.1250218904773
mean 222.63648815771037
std 131.41455746572493
mean 221.85885164928314
std 130.8233945230421
mean 221.39692874484513
std 130.47360705603103
mean 221.07006774711851
std 130.22697062618707
mean 220.78823265711821
std 130.0146663986508
mean 220.55109277283543
std 129.83612395000856
mean 220.37332448113932
std 129.70230043580102
mean 220.24012901902978
std 129.6021441691265
mean 220.12180627673183
std 129.5132267751068
mean 219.97393595164425
std 129.40209840257504
mean 219.82608495034404
std 129.2909554099438
mean 219.64869430810413
st

In [76]:
pprint(outliers[500])

{'id': '0c919abb95',
 'images': [{'id': '795a547950.jpg',
             'url': 'http://assets.epicurious.com/photos/5761ccd8e047a6cf1ee8266d/master/pass/butter-pie-crust.jpg'}],
 'ingredients': [{'text': '2 1/2 cups all purpose flour'},
                 {'text': '1 tablespoon sugar'},
                 {'text': '3/4 teaspoon salt'},
                 {'text': '1 cup (2 sticks) chilled unsalted butter, cut into '
                          '1/2-inch cubes'},
                 {'text': '6 tablespoons (about) ice water'}],
 'instructions': [{'text': 'Mix flour, sugar, and salt in processor.'},
                  {'text': 'Add butter; pulse until coarse meal forms.'},
                  {'text': 'Gradually blend in enough ice water to form moist '
                           'clumps.'},
                  {'text': 'Gather dough into ball; divide in half.'},
                  {'text': 'Form dough into 2 balls; flatten into disks.'},
                  {'text': 'Wrap each in plastic; chill 2 hours or 

In [68]:
outliers[500]["nut"]["ingredients"][0] # should be peanuts but its oil which is why total value is off

{'text': 'oil, corn, peanut, and olive'}

In [78]:
# filter by per recipe (incomplete)
outliers = print_filt(key=lambda r: r["nut"]["nutr_values_per100g"]["energy"] / 100 * sum(r["nut"]["weight_per_ingr"]))

mean 8742.221610160565
std 15556.196298540619
mean 5990.53998506509
std 8638.525940342517
mean 3932.466301739208
std 5037.199288898606
mean 2812.9915009685824
std 3015.2564330429846
mean 2169.1439374383604
std 1949.3421577828385
mean 1819.866176220569
std 1439.8980414077653
mean 1607.972730501549
std 1184.1423682400741
mean 1464.601044287561
std 1031.619998929448
mean 1359.6353096375165
std 927.9216675511376
mean 1284.2340756783462
std 857.7856183368228
mean 1227.7752150950091
std 807.7314984513365
mean 1192.4672512590473
std 777.7940342015638
mean 1162.081501527264
std 752.9013549257305
mean 1136.6809863218575
std 732.2915517725538
mean 1116.1891094619896
std 715.8286806931293
mean 1101.161787242923
std 703.9494072942849
mean 1088.3597057644272
std 693.931771629177
mean 1077.4288201282704
std 685.4363812927522
mean 1071.4084092449675
std 680.7964185951528
mean 1064.50230372048
std 675.5151676392361
mean 1058.5345814850248
std 670.9430100151998
mean 1054.446428145438
std 667.8292629807

In [75]:
r = outliers[100]
print(r["nut"]["nutr_values_per100g"]["energy"] / 100 * sum(r["nut"]["weight_per_ingr"]))
pprint(r)

6014.349999999999
{'id': '028bbd74ea',
 'images': [{'id': 'b681d47161.jpg',
             'url': 'http://www.lovefood.com/images/content/body/oatygingercrunch.jpg'}],
 'ingredients': [{'text': '150 g (5.3oz) plain flour'},
                 {'text': '1.5 tsp baking powder'},
                 {'text': '1.5 tsp ground ginger'},
                 {'text': '75 g (2.6oz) desiccated coconut'},
                 {'text': '165 g (5.8oz) rolled oats'},
                 {'text': '150 g (5.3oz) light brown sugar'},
                 {'text': '150 g (5.3oz) butter'},
                 {'text': '70 g (2.5oz) golden syrup'},
                 {'text': '290 g (10.2oz) icing sugar'},
                 {'text': '1 tbsp ground ginger'},
                 {'text': '120 g (4.2oz) butter'},
                 {'text': '115 g (4.1oz) golden syrup'}],
 'instructions': [{'text': 'Preheat the oven to 180C.'},
                  {'text': 'Lightly grease and line the base and sides of an '
                           '11 x 3